In [59]:
# psh270, jxs535, def456, hkp680

# Task 2
import numpy as np
import pandas as pd
import re
from cleantext.sklearn import CleanTransformer # likely required to ´pip install clean-text´
data = pd.read_csv("https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv")

In [60]:
initial_cleaner = CleanTransformer(fix_unicode=True,               # fix various unicode errors
                                    to_ascii=True,                  # transliterate to closest ASCII representation
                                    lower=True,                     # lowercase text
                                    no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
                                    no_urls=True,                  # replace all URLs with a special token
                                    no_emails=True,                # replace all email addresses with a special token
                                    no_phone_numbers=True,         # replace all phone numbers with a special token
                                    no_numbers=False,               # replace all numbers with a special token
                                    no_digits=False,                # replace all digits with a special token
                                    no_currency_symbols=True,      # replace all currency symbols with a special token
                                    no_punct=True,                 # remove punctuations
                                    replace_with_punct=" ",          # instead of removing punctuations you may replace them
                                    replace_with_url="<url>",
                                    replace_with_email="<email>",
                                    replace_with_phone_number="<phone>",
                                    replace_with_currency_symbol="<cur>",
                                    lang="en"                       # set to 'de' for German special handling
                                    )

general_cleaner = CleanTransformer(fix_unicode=False,               # fix various unicode errors
                                    to_ascii=False,                  # transliterate to closest ASCII representation
                                    lower=False,                     # lowercase text
                                    no_line_breaks=False,           # fully strip line breaks as opposed to only normalizing them
                                    no_urls=False,                  # replace all URLs with a special token
                                    no_emails=False,                # replace all email addresses with a special token
                                    no_phone_numbers=False,         # replace all phone numbers with a special token
                                    no_numbers=True,               # replace all numbers with a special token
                                    no_digits=False,                # replace all digits with a special token
                                    no_currency_symbols=False,      # replace all currency symbols with a special token
                                    no_punct=False,                 # remove punctuations
                                    replace_with_number="<number>",
                                    lang="en"                       # set to 'de' for German special handling
                                    )



In [61]:
three_numb_date = r'(<number> <number> <number>)' #YYYY/MM/DD or DD/MM/YYYY or MM/DD/YYYY
literal_months_date= r'(jan|feb|mar|apr|may|jun|jul|aug|sep|nov|dec)\S* ((<number> ){1,2}|([0-9]{1,2}(st|nd|rd|th)))' #Eg. jun 2nd 2020, january 23. 2021
literal_months_reverse_date = r'((<number> {1,2})|[0-9]{1,2}(st|nd|rd|th)) *(jan|feb|mar|apr|may|jun|jul|aug|sep|nov|dec)\S*' #Eg. 10th february, 4th july
all_dates = (three_numb_date) +'|' + (literal_months_date) +'|'+ (literal_months_reverse_date)
multiple_chars = r'(.)\1{3,}'
special_symbols = r'([^<>a-z ])'#Matches special symbols such as © or ™

In [62]:
#string_test='In gold, the open interest SURPRISINGLY ROSE BY A CONSIDERABLE 9126 CONTRACTS UP TO582,421 WITH THE GOOD SIZED RISE IN PRICE OF GOLD WITH YESTERDAY’S TRADING ($5.55). IN ANOTHER HUGE DEVELOPMENT, WE RECEIVED THE TOTAL NUMBER OF GOLD EFP’S ISSUED FOR WEDNESDAY AND IT TOTALED A HUMONGOUS SIZED 12,223 CONTRACTS OF WHICH FEBRUARY SAW 11,023 CONTRACTS ISSUED AND APRIL SAW THE ISSUANCE OF 1200 CONTRACTS.'
#date_test  = '12/18/10 12/18/2020 12-18-10 12-18-2020 12/18/10 12/18/2020 12.18.10 12.18.2020 noise 12182010 december 18, 2010 janu 10th march 1st 3st january Dekjkj 10th  noise 10/20  noise noise 2020 10th january 2021'

def clean_column(data, col_name):
    for i, entry in enumerate(data[col_name]):    
        #We first convert to lower case and replace punctuation with space such that dates can
        #more easily be processed (eg. 10.12.2020 -> 10 12 2020 -> <NUMBER> <NUMBER> <NUMBER> instead of <NUMBER><NUMBER><DIGIT> or something)
        cleaned = initial_cleaner.transform([entry])[0]
        cleaned = general_cleaner.transform([cleaned])[0]
        cleaned = re.sub(all_dates, '<date> ', cleaned)
        cleaned = re.sub(special_symbols,'',cleaned)
        cleaned = re.sub(multiple_chars, '', cleaned)
        data.at[i, col_name] = cleaned

def clean_data(data):
    #Dropping unneeded columns
    cols_to_delete = ["Unnamed: 0","id","scraped_at","inserted_at","updated_at"]
    for column in data.columns:
        if data[column].isnull().values.all():
            cols_to_delete.append(column)

    data.drop(cols_to_delete, 1, inplace=True)
    
    clean_column(data, "content")
    clean_column(data, "title")
    return data


In [63]:
data["content"][4]

'Donald Trump has the unnerving ability to ability to create his own reality and convince millions of Americans that what he says it is true. The problem with the president lying is that he then believes his own lies. A new poll shows how that can get the country into deep trouble.\n\nThe new ABC News/Washington Post poll came out after the president’s physician gave him a physical and mental exam. The doctor gave Trump a clean bill of health, added an inch to his height, and claimed he was fit to serve for seven more years.\n\nThis poll was able to capture Americans’ opinions after a new book came out indicating that people around Trump questioned his emotional stability and ability to hold office. In addition, the new poll gave the respondents the time to hear Trump tell the public that he was a “very stable genius” before they were interviewed. He said:\n\n‘Actually, throughout my life, my two greatest assets have been mental stability and being, like, really smart.’\n\nThe ABC/Wash

Since we are working on a subset of the full dataset, there is no need to include the old index/ID.
Furthermore, since pandas adds an index column, we do not have a need for the already existing (possibly error prone) local index (Unnamed: 0).

Metadata regarding scraping, time of updates etc. do not serve a significant impact on the processing we wish to perform.

In [64]:
pd.set_option('display.max_rows', None)
data = clean_data(data)
data
df = pd.DataFrame(data, columns=['domain','type','url','content','title','authors', 'meta_keywords', 'meta_description', 'tags'])
df.sort_values(by=['type'], ascending=False)

C:\Users\Peter\AppData\Local\Temp\ipykernel_22240\2449131707.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data.drop(cols_to_delete, 1, inplace=True)


,domain,type,url,content,title,authors,meta_keywords,meta_description,tags
0,awm.com,unreliable,http://awm.com/church-congregation-brings-gift...,sometimes the power of christmas will make you...,church congregation brings gift to waitresses ...,Ruth Harris,[''],NaN,NaN
5,blackagendareport.com,unreliable,https://blackagendareport.com/articlelist/Medi...,republicans and democrats alike are willing to...,black agenda report,"Margaret Kimberley, Bar Editor, Senior Columni...",[''],NaN,NaN
201,anonhq.com,unreliable,http://anonhq.com/in-2017-police-killed-1184-1...,the new year is a new beginning full of optimi...,in <number> police killed <number> times the n...,Please Enter Your Name Here,[''],"149 of those killed were completely unarmed, w...",NaN
16,breakpoint.org,unreliable,http://breakpoint.org/category/signs-and-wonde...,signs and wonders snap out of it the bible is ...,signs and wonders archives,Warren Cole Smith,[''],NaN,"Human Rights & Persecution, Marriage & Family,..."
3,awm.com,unreliable,http://awm.com/elusive-alien-of-the-sea-caught...,when a rare shark was caught scientists were l...,elusive alien of the sea caught by scientist f...,Alexander Smith,[''],NaN,NaN
2,cnnnext.com,unreliable,http://www.cnnnext.com/video/18526/never-hike-...,never hike alone a friday the th fan film usa ...,never hike alone a friday the th fan film full...,NaN,[''],Never Hike Alone: A Friday the 13th Fan Film ...,NaN
169,undergroundhealth.com,unknown,https://www.undergroundhealth.com/best-type-ma...,in our modern world it is very common for peop...,best type of mattress for herniated discs,NaN,[''],"In our modern world, it is very common for peo...",NaN
222,strategic-culture.org,unknown,https://www.strategic-culture.org/pview/2017/1...,for the first time since the end of the cold w...,nato defense chiefs and us lawmakers take new ...,NaN,"['United States', 'Trump', 'INF Treaty', 'NDAA...",The NATO defense chiefs and US lawmakers have ...,NaN
223,strategic-culture.org,unknown,https://www.strategic-culture.org/news/2013/02...,the return of empires ii part i a world of pos...,the return of empires ii,NaN,[''],NaN,"Chatham House, Hague Tribunal, Sharp, Five Eye..."
218,strategic-culture.org,unknown,https://www.strategic-culture.org/news/2017/01...,is vietnam tilting toward china xuan loc doan ...,is vietnam tilting toward china,NaN,[''],NaN,"Chatham House, Hague Tribunal, Sharp, Five Eye..."


In [65]:
print(data["title"][4])

trump s genius poll is complete the results have americans bursting with laughter


In [66]:
data["content"][4]

'donald trump has the unnerving ability to ability to create his own reality and convince millions of americans that what he says it is true the problem with the president lying is that he then believes his own lies a new poll shows how that can get the country into deep trouble the new abc news washington post poll came out after the president s physician gave him a physical and mental exam the doctor gave trump a clean bill of health added an inch to his height and claimed he was fit to serve for seven more years this poll was able to capture americans opinions after a new book came out indicating that people around trump questioned his emotional stability and ability to hold office in addition the new poll gave the respondents the time to hear trump tell the public that he was a very stable genius before they were interviewed he said actually throughout my life my two greatest assets have been mental stability and being like really smart the abc washington post poll discovered that 

In [67]:
# Task 3
# things to explore
# To be fake news it has to not have meta keywords or description
# All fake news originates from beforeitsnews.com	
# 1: clickbait type vs titles
# 2: word frequency n-gram vs article type
# 3: authors changing type

In [68]:
# Task 4
group_nr = 14
group_substring_raw = "ABCDEFGHIJKLMNOPRSTUVWZABCDEFGHIJKLMNOPRSTUVWZ"[group_nr%23:group_nr%23+10]
group_substring = ""

for letter in np.sort(list(group_substring_raw)):
    group_substring += letter
    
group_substring

'AOPRSTUVWZ'

In [69]:
import requests
from bs4 import BeautifulSoup

response = requests.get('https://en.wikinews.org/wiki/Category:Politics_and_conflicts')
contents = response.text

soup = BeautifulSoup(contents, 'html.parser')

ModuleNotFoundError: No module named 'bs4'

In [ ]:
cont = soup.find_all('a', class_="external text")
links = []
for line in cont:
    if np.isin((line.attrs["href"][-1]), list(group_substring)):
        links.append(line.attrs["href"])

links = [links[0]] # Go through each href and scrape untill next h3 letter is hit. TODO
for link in links:
    response = requests.get(link)
    contents = response.text

    soup = BeautifulSoup(contents, 'html.parser')

In [ ]:
cont = soup.find_all("div", id="mw-pages")